# Generating Email Responses Using Amazon Titan Model on Bedrock

In this lab, you learn how to use Large Language Model (LLM) to generate an email response to a customer who provided negative feedback on the quality of customer service they received from the support engineer. In this notebook, you generate an email with a thank you note based on the customer's previous email. You use the Amazon Titan model using the Amazon Bedrock API with Boto3 client.

The prompt used in this task is called a zero-shot prompt. In a zero-shot prompt, you describe the task or desired output to the language model in plain language. The model then uses its pre-trained knowledge and capabilities to generate a response or complete the task based solely on the provided prompt.

#### Scenario
You are an AI/ML Engineer at K21Technologies, and your company receives a significant amount of customer feedback, including both positive and negative responses regarding the support provided by your customer service team. One of your tasks is to generate personalized and empathetic responses to customers who have expressed dissatisfaction, aiming to address their concerns and improve customer satisfaction. Using a large language model (LLM), you need to automate the process of drafting responses based on their sentiment from previous communications.

## 1. Environment setup

In this task, you set up your environment.

In [ ]:
# Importing necessary libraries to work with AWS, files, and environment variables
import json  # For working with JSON data
import os  # For interacting with the operating system (e.g., file paths)
import sys  # For modifying system paths

# Importing Boto3 (AWS SDK for Python) and botocore (handles low-level AWS interactions)
import boto3
import botocore

# Adding the parent directory to the system path so we can import files from there
module_path = ".."  # This points to the parent directory
sys.path.append(os.path.abspath(module_path))  # Adds the parent directory to the search path for modules

# Create a connection to Amazon Bedrock service using the Boto3 client
# 'bedrock-runtime' specifies the service we want to connect to
# The region_name is fetched from the environment variable AWS_DEFAULT_REGION
bedrock_client = boto3.client(
    'bedrock-runtime',  # Service name: Bedrock
    region_name=os.environ.get("AWS_DEFAULT_REGION", None)  # AWS region, if available from environment settings
)


## 2. Generate text

In this task, you prepare an input for the Amazon Bedrock service to generate an email.

In [ ]:
# Define the prompt to be sent to the AI model for generating an email
# This is the instruction that tells the model what to do
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, AnyCompany to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""


In [ ]:
# Prepare the data to send to the AI model in JSON format
body = json.dumps({
    "inputText": prompt_data,  # The prompt we created earlier (email request)
    "textGenerationConfig": {  # Settings for generating the text
        "maxTokenCount": 8192,  # Max length of the response
        "stopSequences": [],  # No specific stop point
        "temperature": 0,  # Predictable, less random response
        "topP": 0.9  # Controls how varied the response can be
    }
})


## 3. Invoke the Amazon Titan Large language model

In this task, you explore how the model generates an output based on the prompt created earlier.

### Complete Output Generation

This email is generated using the Amazon Titan model by understanding the input request and utilizing its inherent understanding of different modalities. The request to the API is synchronous and waits for the entire output to be generated by the model.

In [ ]:
# Set the model ID and request headers
modelId = 'amazon.titan-text-express-v1'  # Specify the model version to use
accept = 'application/json'  # The expected response format
contentType = 'application/json'  # The format of the request body
outputText = "\n"  # Initialize an empty string to store the response

try:
    # Invoke the AI model using the Bedrock client
    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    
    # Read and parse the response from the model
    response_body = json.loads(response.get('body').read())
    
    # Extract the generated output text from the response
    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    # Handle specific errors returned by AWS
    if error.response['Error']['Code'] == 'AccessDeniedException':
        # If access is denied, print an error message with troubleshooting links
        print(f"\x1b[41m{error.response['Error']['Message']}\n"
              "To troubleshoot this issue, please refer to the following resources.\n"
              "https://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\n"
              "https://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
    else:
        # For any other errors, re-raise the error
        raise error



In [ ]:
# The relevant part of the response starts after the first newline character '\n'.
# Extract the email text by slicing the response string from after the first newline.

email = outputText[outputText.index('\n')+1:]  # Find the first newline and get everything after it
print(email)  # Print the generated email



### Streaming Output Generation

Bedrock also supports that the output can be streamed as it is generated by the model in form of chunks. This email is generated by invoking the model with streaming option. `invoke_model_with_response_stream` returns a `ResponseStream` which you can read from.

In [ ]:
# Initialize an empty list to store the output chunks
output = []

try:
    # Invoke the model with a response stream to get results as chunks
    response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')  # Get the response stream (body)

    i = 1  # Counter for chunk numbers
    if stream:
        # Process each chunk of data from the stream
        for event in stream:
            chunk = event.get('chunk')  # Get the chunk of data from the event
            if chunk:
                # Convert the chunk from bytes to a JSON object
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']  # Extract the generated text (email)
                output.append(text)  # Add the chunk to the output list
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')  # Print the current chunk
                i += 1  # Increment the chunk counter

except botocore.exceptions.ClientError as error:
    # Handle specific AWS access errors
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\n"
              "To troubleshoot this issue, please refer to the following resources.\n"
              "https://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\n"
              "https://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
    else:
        raise error  # Re-raise other errors


The stream with response approach helps to quickly obtain the output of the model and allows the service to complete it as you read. This assists in use cases where you request the model to generate longer pieces of text. You can later combine all the chunks generated to form the complete output and use it for your use case. 

In [ ]:
# Combine all the chunks of text into a single string
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')  # Print a header indicating the final output

# Join all the pieces of text from the output list into one complete string
complete_output = ''.join(output)

# Print the complete generated email
print(complete_output)



You have now experimented with using the boto3 SDK, which provides basic exposure to the Amazon Bedrock API. Using this API, you have seen the use case of generating an email to respond to a customer's negative feedback.

